# Strategy V2

In analysis, we found that all instruments except 26 followed GBM-like properties. This means 
they are trending/momentum instruments and we aim to implement, test and validate the following 
trading strategies to see which gives the best profit factor, sharpe ratio and competition score:

- Donchian Breakout
- EMA Crossovers

Each strategy will also have an optimised, grid-searched lookback(s).

In [31]:
import pandas as pd
import numpy as np
import copy

from pandas import DataFrame, Series
from typing import List, Dict
from numpy import ndarray

position_limits: int = 10000
allocated_instruments: List[int] = []
for i in range(0,50): allocated_instruments.append(i)
commission_fee: float = 0.0005

We first load in the price_history

In [32]:
def get_price_history() -> Dict[int, DataFrame]:
	# Get Price Data
	raw_prices: DataFrame = pd.read_csv("../../prices.txt", sep=r"\s+", index_col=None,
		header=None)
	price_data: ndarray = raw_prices.values[:500,:].T

	# Initialise Price History
	price_history: Dict[int, DataFrame] = {}

	# Assign price data to insturments
	for instrument_no in allocated_instruments:
		instrument_price_history: Dict[str, ndarray] = {}
		instrument_price_history["price"] = price_data[instrument_no]
		price_history[instrument_no] = pd.DataFrame(instrument_price_history)


	return price_history

### Strategies

**Donchian Breakout**

In [33]:
def donchian_breakout(price_history: Dict[int, DataFrame], lookbacks: Dict[int, int]) -> Dict[int,
	DataFrame]:
	for instrument_no in allocated_instruments:
		# Assign upper and lower band
		price_history[instrument_no]["upper"] = price_history[instrument_no]["price"].rolling(
				lookbacks[instrument_no] - 1).max().shift(1)
		price_history[instrument_no]["lower"] = price_history[instrument_no]["price"].rolling(
				lookbacks[instrument_no] - 1).min().shift(1)

		# Assign signals
		price_history[instrument_no]["signal"] = np.nan
		price_history[instrument_no].loc[price_history[instrument_no]["price"] > price_history[
			instrument_no]["upper"], 'signal']	= 1
		price_history[instrument_no].loc[price_history[instrument_no]["price"] < price_history[
			instrument_no]["lower"], 'signal']	= -1

		price_history[instrument_no]["signal"] = price_history[instrument_no]["signal"].ffill()

	return price_history

**MA Crossover**

In [34]:
def ma_crossover(price_history: Dict[int, DataFrame], lookbacks: Dict[int, List[int]]) -> (
	Dict)[int, DataFrame]:
	for instrument_no in allocated_instruments:
		# Get fast and slow MA
		fast_ma: ndarray = price_history[instrument_no]["price"].ewm(
			span=lookbacks[instrument_no][0], adjust=False).mean()
		slow_ma: ndarray = price_history[instrument_no]["price"].ewm(
			span=lookbacks[instrument_no][1], adjust=False).mean()
		
		price_history[instrument_no]["signal"] = np.nan
		price_history[instrument_no].loc[fast_ma > slow_ma, 'signal'] = 1
		price_history[instrument_no].loc[fast_ma < slow_ma, 'signal'] = -1
		price_history[instrument_no]["signal"] = price_history[instrument_no]["signal"].ffill()
	
	return price_history

### Backtesting

In [35]:
def get_strategy_results(price_history: Dict[int, DataFrame]) -> Dict[int, DataFrame]:
	for instrument_no in allocated_instruments:
		# Get Log Returns
		price_history[instrument_no]["log_return"] = np.log(price_history[instrument_no]
		["price"]).diff().shift(-1)

		# Get Strategy Return
		price_history[instrument_no]["strategy_return"] = (price_history[instrument_no]["signal"]
														   * price_history[instrument_no]["log_return"])

		# Get Position changes
		position_change: ndarray = price_history[instrument_no]["signal"].diff().abs()

		# Apply the commission fee
		price_history[instrument_no]["strategy_return"] -= position_change * commission_fee


	return price_history

def show_performance_metrics(strategy_results: Dict[int, DataFrame]) -> None:
	performance_metrics: Dict[str, List[int | float]] = {}
	performance_metrics["Instrument No."] = allocated_instruments
	performance_metrics["Profit Factor"] = []
	performance_metrics["Sharpe Ratio"] = []
	
	for instrument_no in allocated_instruments:
		# Get Returns
		returns: Series = strategy_results[instrument_no]["strategy_return"]

		# Compute performance metrics
		profit_factor = returns[returns > 0].sum() / returns[returns < 0].abs().sum()
		sharpe = (returns.mean() / returns.std()) * (252 ** 0.5)
		
		performance_metrics["Profit Factor"].append(profit_factor)
		performance_metrics["Sharpe Ratio"].append(sharpe)
	
	performance_metrics_df: DataFrame = pd.DataFrame(performance_metrics)
	print(performance_metrics_df.to_string(index=False))
		


### Grid Searching (Optimise for sharpe ratio)

In [36]:
def get_donchian_breakout_lookbacks() -> Dict[int, int]:
	lookbacks: Dict[int, int] = {}
	best_sharpes: Dict[int, float] = {
		instrument_no: -100000000.0 for instrument_no in allocated_instruments
	}
	
	price_history: Dict[int, DataFrame] = get_price_history()
	
	for lookback in range(30, 201):
		current_lookback: Dict[int, int] = {instrument_no: lookback for instrument_no in 
			allocated_instruments}
		strategy_output: Dict[int, DataFrame] = donchian_breakout(copy.deepcopy(price_history), 
			current_lookback)
		strategy_results: Dict[int, DataFrame] = get_strategy_results(strategy_output)
		
		for instrument_no in allocated_instruments:
			# Get Returns
			returns: Series = strategy_results[instrument_no]["strategy_return"]
			
			# Compute Sharpe
			sharpe = (returns.mean() / returns.std()) * (252 ** 0.5)
			
			# If sharpe is better than current sharpe, replace lookback
			if sharpe > best_sharpes[instrument_no]:
				best_sharpes[instrument_no] = sharpe
				lookbacks[instrument_no] = lookback
	
	return lookbacks

def get_ma_crossover_lookbacks() -> Dict[int, List[int]]:
	lookbacks: Dict[int, List[int]] = {instrument_no: [] for instrument_no in allocated_instruments}
	best_sharpes: Dict[int, float] = {
		instrument_no: -10000000.0 for instrument_no in allocated_instruments
	}
	
	price_history: Dict[int, DataFrame] = get_price_history()
	
	for slow_lookback in range(50, 201):
		for fast_lookback in range(5, 11):
			current_lookback: Dict[int, List[int]] = {
				instrument_no: [fast_lookback, slow_lookback] for instrument_no in allocated_instruments
			}
			strategy_output: Dict[int, DataFrame] = ma_crossover(copy.deepcopy(price_history),
				current_lookback)
			strategy_results: Dict[int, DataFrame] = get_strategy_results(strategy_output)
			
			for instrument_no in allocated_instruments:
				returns: Series = strategy_results[instrument_no]["strategy_return"]
				
				sharpe = (returns.mean() / returns.std()) * (252 ** 0.5)
				
				if sharpe > best_sharpes[instrument_no]:
					best_sharpes[instrument_no] = sharpe
					lookbacks[instrument_no] = [fast_lookback, slow_lookback]
	
	return lookbacks

### Performance Metrics


In [37]:
# Get optimised lookbacks
donchian_breakout_lookbacks: Dict[int, int] = get_donchian_breakout_lookbacks()
ma_crossover_lookbacks: Dict[int, List[int]] = get_ma_crossover_lookbacks()

price_history: Dict[int, DataFrame] = get_price_history()
db_output: Dict[int, DataFrame] = donchian_breakout(copy.deepcopy(price_history), 
	donchian_breakout_lookbacks)
ma_cross_output: Dict[int, DataFrame] = ma_crossover(copy.deepcopy(price_history), 
	ma_crossover_lookbacks)

db_results: Dict[int, DataFrame] = get_strategy_results(db_output)
ma_cross_results: Dict[int, DataFrame] = get_strategy_results(ma_cross_output)
show_performance_metrics(db_results)
print(donchian_breakout_lookbacks)

show_performance_metrics(ma_cross_results)
print(ma_crossover_lookbacks)

 Instrument No.  Profit Factor  Sharpe Ratio
              0       1.156975      0.929310
              1       1.208946      1.218807
              2       1.228547      1.322497
              3       1.082080      0.491369
              4       1.261922      1.453206
              5       1.344871      1.904097
              6       1.164285      0.957696
              7       1.069305      0.420549
              8       0.988750     -0.072708
              9       1.145569      0.839035
             10       1.152185      0.881011
             11       1.103274      0.620036
             12       1.162521      0.939399
             13       1.139048      0.835589
             14       1.175865      0.995621
             15       1.065247      0.387051
             16       1.103871      0.623333
             17       1.109285      0.666689
             18       0.940731     -0.386133
             19       1.132940      0.784243
             20       1.317690      1.726149
          

We can conclude that we can use Donchian Breakouts for assets 1, 2, 4, 5, 23 and 27 and EMA 
Crossovers on asset 0